## MBA em Ciência de Dados
# Redes Neurais e Arquiteturas Profundas

### <span style="color:darkred">Módulo 6 - Redes neurais para dados sequenciais</span>

#### <span style="color:darkred">**Parte 2: Word Embedding**</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

from tensorflow import keras
from keras import layers
from numpy.random import seed
from tensorflow.random import set_seed

## Carregando representações pré-treinadas em português 

Após salvar, vamos montar um dicionário com palavra/vetor

In [2]:
#!wget http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s50.zip

In [3]:
#!mv download.php?file=embeddings%2Fglove%2Fglove_s50.zip glove_s50.zip
#!unzip -q glove_s50.zip

s50 : 50 dimensões por palavra

In [13]:
# path_to_glove_file = os.path.join(
#     os.path.expanduser("~"), "/content/glove_s50.txt" # colab
# )

path_to_glove_file = "./glove_s50.txt"  # jupyter
# path_to_glove_file = "glove_s50.txt"  # jupyter

embeddings_index = {}
with open(path_to_glove_file, encoding="utf-8", mode='r') as f: 
# with open(path_to_glove_file, encoding="utf-8", errors = 'backslashreplace') as f:
# with open(path_to_glove_file, encoding="utf-8") as f:
# with open(path_to_glove_file, encoding="latin1") as f:    
# with open(path_to_glove_file, encoding="cp1252") as f:
# with open(path_to_glove_file, encoding="ISO-8859-1") as f:
# with open(path_to_glove_file) as f:   
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Encontrados %s word vectors." % len(embeddings_index))

<ipython-input-13-70b59e83ceaf>:18: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  coefs = np.fromstring(coefs, "f", sep=" ")


Encontrados 929594 word vectors.


In [4]:
# vetores da palavra aprovação
print(embeddings_index['aprovação'])
print(len(embeddings_index['aprovação']))

[ 6.984870e-01  1.938170e-01  1.839920e-01 -2.590166e+00 -3.155430e-01
 -1.469410e-01  1.290320e-01  3.814410e-01 -4.846610e-01  3.721310e-01
  6.471990e-01 -1.248160e+00 -3.151210e-01  3.676890e-01 -7.965720e-01
  2.589710e-01 -1.260200e-02 -6.782460e-01 -4.735670e-01  3.739230e-01
  1.437597e+00  2.001800e-02  9.999200e-02 -1.829620e-01  2.779400e-01
  1.222500e-01 -2.345070e-01 -7.791430e-01  6.422940e-01  3.167230e-01
 -3.914640e-01  3.333300e-01  2.291640e-01 -9.465310e-01 -2.157560e-01
 -3.246800e-02 -3.029230e-01  9.146800e-02 -1.788646e+00 -2.995630e-01
 -3.183580e-01 -7.586490e-01  2.524000e-03 -6.656960e-01  7.843900e-01
  1.341660e-01  6.273990e-01  3.014050e-01 -4.354190e-01  1.121057e+00]
50


## Base de dados: rumor Brazil 2018

Base de dados de texto para classificação em frases "falsas" ou "verdadeiras" conforme checado por agências

In [5]:
df = pd.read_csv("data/rumor-election-brazil-2018.csv", delimiter=';')
texto = df['texto']
rotulos = (df['rotulo']=='VERDADE').astype(int)

class_names = ["FALSO", "VERDADEIRO"]

print(texto[:10])
print(rotulos[:10])

0    Salário Mínimo: R$ 950,00. Bolsa Presidiário: ...
1    Empresa contratada pelo TSE para apuração dos ...
2    O Aloizio Mercadante, ministro da Educação, mo...
3    Há um complô espalhando fake news descaradas e...
4    Somente em 2017, mais de 800 milhões de tonela...
5    Nunca vi o Lula pronunciar essa palavra fascis...
6    O Mourão, por exemplo, foi ele próprio tortura...
7    O PSB, todos os seus governadores e o seu pres...
8    Bolsonaro Nunca aprovou um projeto de seguranç...
9    Ele Lula não pode aparecer mais que 25% no hor...
Name: texto, dtype: object
0    0
1    0
2    0
3    0
4    1
5    0
6    0
7    0
8    1
9    1
Name: rotulo, dtype: int32


In [6]:
counts = np.bincount(rotulos)
print(counts)

[239 221]


In [7]:
rng = np.random.RandomState(1)
rng.shuffle(texto)
rng = np.random.RandomState(1)
rng.shuffle(rotulos)

validation_split = 0.1
num_validation = int(validation_split * len(texto))
x_train = texto[:-num_validation]
x_val = texto[-num_validation:]
y_train = rotulos[:-num_validation]
y_val = rotulos[-num_validation:]

<ipython-input-7-87b77ae273f0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rng.shuffle(texto)


Vocabulário irá considerar até 20 mil palavras, e irá truncar sequências com mais de 32 tokens

tokens = palavras

In [8]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=32)
text_ds = tf.data.Dataset.from_tensor_slices(x_train).batch(16)
vectorizer.adapt(text_ds)

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 0: unexpected end of data

Podemos ver o resultado do vetorizador, que exibe 1 para palavras fora do vocabulário

In [10]:
output = vectorizer([["um dois one two three"]])
output.numpy()[0, :5]

array([19, 92,  1,  1,  1])

Abaixo, considerando o número de tokens (palavras) da base de dados, vamos tentar gerar seus embedding vectors.

Palavras fora do vocabulário não são convertidas. Comumente artigos, números e outros.

In [11]:
num_tokens = len(voc) + 2
print("Número de tokens: ", num_tokens)
embedding_dim = 50
convertidas = 0
falhas = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
print(embedding_matrix.shape)
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        if (embedding_vector.shape[0] != embedding_dim):
          falhas += 1
        else:
          # Words not found in embedding index will be all-zeros.
          # This includes the representation for "padding" and "OOV"
          embedding_matrix[i] = embedding_vector
          convertidas += 1
    else:
        falhas += 1

print("Palavras convertidas: %d / não convertidas: %d)" % (convertidas, falhas))

Número de tokens:  1944
(1944, 50)
Palavras convertidas: 1785 / não convertidas: 157)


Montando a base de dados para o treinamento, no formato numpy array

Cada sentença tem no máximo 32 tokens (limite fixado no código acima): output_sequence_length=32

In [12]:
x_train = vectorizer(np.array([[s] for s in x_train])).numpy()
x_val = vectorizer(np.array([[s] for s in x_val])).numpy()

y_train = np.array(y_train)
y_val = np.array(y_val)

print(x_train.shape)
print(x_val.shape)

(414, 32)
(46, 32)


## Camada de "Word Embedding"

Para isso devemos informar o número de palavras, a dimensionalidade e passar a matrix com pesos pré-treinados.

Caso não utilizemos os parâmetros `embeddings_initializer`, os pesos serão aleatórios.

Também deixamos essa camada não treinável pois não queremos modificar as representações 

In [13]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [14]:
# decaimento de learning rate
def scheduler(epoch, lr):
    if (epoch % 5 == 0): print("Época %04d - learning rate %.9f" % (epoch, lr))
    if (epoch+1 < 5):
        return lr
    else:
        return np.round(lr * tf.math.exp(-0.005),9)

callbacklr = tf.keras.callbacks.LearningRateScheduler(scheduler)

### Modelo baseado em Convoluções

In [28]:
from tensorflow.keras import layers

seed(1)
set_seed(2)

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(64, 2, activation="relu", padding="same")(embedded_sequences)
x = layers.Conv1D(128, 3, strides=2, activation="relu", padding="same")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(1, activation="sigmoid")(x)
modelConv = keras.Model(int_sequences_input, preds)
modelConv.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          97200     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, None, 64)          6464      
_________________________________________________________________
conv1d_9 (Conv1D)            (None, None, 128)         24704     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0   

In [29]:
modelConv.compile(
    loss="binary_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["acc"]
)
modelConv.fit(x_train, y_train, batch_size=16, epochs=25, 
                  callbacks=[callbacklr], validation_data=(x_val, y_val))

Epoch 1/25
Época 0000 - learning rate 0.001000000
26/26 [==============================] - 1s 7ms/step - loss: 0.7456 - acc: 0.4662 - val_loss: 0.6886 - val_acc: 0.5000
Epoch 2/25
26/26 [==============================] - 0s 4ms/step - loss: 0.6763 - acc: 0.5845 - val_loss: 0.6802 - val_acc: 0.5435
Epoch 3/25
26/26 [==============================] - 0s 4ms/step - loss: 0.6327 - acc: 0.6812 - val_loss: 0.6669 - val_acc: 0.6739
Epoch 4/25
26/26 [==============================] - 0s 4ms/step - loss: 0.5970 - acc: 0.7029 - val_loss: 0.6576 - val_acc: 0.6522
Epoch 5/25
26/26 [==============================] - 0s 4ms/step - loss: 0.5438 - acc: 0.7609 - val_loss: 0.6579 - val_acc: 0.6522
Epoch 6/25
Época 0005 - learning rate 0.000995013
26/26 [==============================] - 0s 4ms/step - loss: 0.4692 - acc: 0.7657 - val_loss: 0.6720 - val_acc: 0.5870
Epoch 7/25
26/26 [==============================] - 0s 4ms/step - loss: 0.4378 - acc: 0.8213 - val_loss: 0.6631 - val_acc: 0.6522
Epoch 8/25
2

Podemos agora testar frases novas!

Para isso vou criar uma camada de entrada que passa por um vetorizador, e depois alimenta o modelo.

In [30]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = modelConv(x)
end_to_end_model = keras.Model(string_input, preds)

frase = "Na pós graduação, as mulheres são maioria"
classe = (end_to_end_model.predict([[frase]])[0] > 0.5).astype(int)
print(frase, ': ', class_names[classe[0]])

frase = "As queimadas esse ano são equivalentes a uma área do tamanho do Reino Unido"
classe = (end_to_end_model.predict([[frase]])[0] > 0.5).astype(int)
print(frase, ': ', class_names[classe[0]])

frase = "Acabou a corrupção no Brasil"
classe = (end_to_end_model.predict([[frase]])[0] > 0.5).astype(int)
print(frase, ': ', class_names[classe[0]])

frase = "Para poder ganhar eleições, presidente faz aliança com partidos grandes"
classe = (end_to_end_model.predict([[frase]])[0] > 0.5).astype(int)
print(frase, ': ', class_names[classe[0]])

Na pós graduação, as mulheres são maioria :  VERDADEIRO
As queimadas esse ano são equivalentes a uma área do tamanho do Reino Unido :  VERDADEIRO
Acabou a corrupção no Brasil :  VERDADEIRO
Para poder ganhar eleições, presidente faz aliança com partidos grandes :  VERDADEIRO


### Modelo utilizando GRU

Vamos substituir uma camada convolucional por uma GRU (poderia ser uma LSTM também)

In [21]:
seed(1)
set_seed(2)

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(64, 2, activation="relu", padding="same")(embedded_sequences)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.5)(x)
predsGRUout = layers.Dense(1, activation="sigmoid")(x)
modelGRU = keras.Model(inputs=int_sequences_input, outputs=predsGRUout)
modelGRU.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          97200     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 64)          6464      
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                24960     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65  

In [22]:

modelGRU.compile(
    loss="binary_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["acc"]
)
modelGRU.fit(x_train, y_train, batch_size=16, epochs=25, 
                  callbacks=[callbacklr], validation_data=(x_val, y_val))

Epoch 1/25
Época 0000 - learning rate 0.001000000
26/26 [==============================] - 2s 15ms/step - loss: 0.6929 - acc: 0.4952 - val_loss: 0.7073 - val_acc: 0.5435
Epoch 2/25
26/26 [==============================] - 0s 5ms/step - loss: 0.6861 - acc: 0.4952 - val_loss: 0.6992 - val_acc: 0.4348
Epoch 3/25
26/26 [==============================] - 0s 5ms/step - loss: 0.6832 - acc: 0.5217 - val_loss: 0.7058 - val_acc: 0.5435
Epoch 4/25
26/26 [==============================] - 0s 5ms/step - loss: 0.6735 - acc: 0.5217 - val_loss: 0.7049 - val_acc: 0.4348
Epoch 5/25
26/26 [==============================] - 0s 5ms/step - loss: 0.6712 - acc: 0.5242 - val_loss: 0.7069 - val_acc: 0.4783
Epoch 6/25
Época 0005 - learning rate 0.000995013
26/26 [==============================] - 0s 5ms/step - loss: 0.6658 - acc: 0.5386 - val_loss: 0.7039 - val_acc: 0.4565
Epoch 7/25
26/26 [==============================] - 0s 5ms/step - loss: 0.6543 - acc: 0.5556 - val_loss: 0.7103 - val_acc: 0.3913
Epoch 8/25


In [23]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
predsGRU = modelGRU(x)
end_to_end_modelGRU = keras.Model(string_input, predsGRU)

frase = "Na pós graduação, as mulheres são maioria"
classe = (end_to_end_modelGRU.predict([[frase]])[0] > 0.5).astype(int)
print(frase, ': ', class_names[classe[0]])

frase = "As queimadas esse ano são equivalentes a uma área do tamanho do Reino Unido"
classe = (end_to_end_modelGRU.predict([[frase]])[0] > 0.5).astype(int)
print(frase, ': ', class_names[classe[0]])

frase = "Acabou a corrupção no Brasil"
classe = (end_to_end_modelGRU.predict([[frase]])[0] > 0.5).astype(int)
print(frase, ': ', class_names[classe[0]])

frase = "Para poder ganhar eleições, presidente faz aliança com partidos grandes"
classe = (end_to_end_modelGRU.predict([[frase]])[0] > 0.5).astype(int)
print(frase, ': ', class_names[classe[0]])

Na pós graduação, as mulheres são maioria :  VERDADEIRO
As queimadas esse ano são equivalentes a uma área do tamanho do Reino Unido :  FALSO
Acabou a corrupção no Brasil :  FALSO
Para poder ganhar eleições, presidente faz aliança com partidos grandes :  VERDADEIRO
